In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
from scipy.stats import uniform, randint
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV, learning_curve, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pickle

## Loading dataset

In [ ]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv', delimiter=',', header=0)
df = pd.DataFrame(df)
df.head()

In [ ]:
df["DEATH_EVENT"].value_counts().plot(kind="bar")
plt.title("Class Distribution")
plt.savefig('class_dristribution.png', dpi=300)
plt.show()

### Loading types

In [108]:
def types_loading(df):
    global X, y, feature_names, X_train, X_test, Y_train, Y_test
    
    df['sex'] = df['sex'].astype("category")
    df['smoking'] = df['smoking'].astype("category")
    df['high_blood_pressure'] = df['high_blood_pressure'].astype("category")
    df['diabetes'] = df['diabetes'].astype("category")
    df['anaemia'] = df['anaemia'].astype("category")
    
    # Removal of time variable due to date leak
    df = df.drop(['time'], axis=1)  
    
    cor = df.corr() 
    corr_target = abs(cor["DEATH_EVENT"])
    
    # Removal of low-correlated variables
    relevant_features = corr_target[corr_target>0.07]
    df =  df[relevant_features.index.tolist()]

    X = df.drop(['DEATH_EVENT'], axis=1)  
    y = df['DEATH_EVENT']

    feature_names = X.columns.tolist()
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

### XGBoost + RFECV + RandomizedSearchCV

In [152]:
def xgb_rfecv(scoring_metric, imbalanced_data):
    global best_params, grid_search, xgb_clf_best, removed_features, rfecv, X_train_rfe, explainer, shap_values, X_test_rfe, cm

    xgb_clf = XGBClassifier(scoring_metric=scoring_metric, enable_categorical=True, objective="binary:logistic")
    
    cv = StratifiedKFold(n_splits=5)
    rfecv = RFECV(estimator=xgb_clf, cv=cv,step=1, scoring="f1", n_jobs=-1)
    rfecv.fit(X_train, Y_train)
    
    X_train_rfe = rfecv.transform(X_train)
    X_test_rfe = rfecv.transform(X_test)
    
    selected_features = [feature for feature, selected in zip(feature_names, rfecv.support_) if selected]
    print("Selected features:", selected_features)
    
    param_grid = {
        'n_estimators': randint(50, 300),
        'max_depth': randint(3, 9),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'alpha': uniform(0, 1),
        'lambda': uniform(0, 1),
        'min_child_weight': randint(1, 10),
        'gamma': uniform(0, 1)
    }

    param_grid['scoring_metric'] = [scoring_metric]
    
    if imbalanced_data:
        scale_pos_weight = float(Y_train.value_counts()[0]) / Y_train.value_counts()[1]
        param_grid['scale_pos_weight'] = [scale_pos_weight]

    randomized_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_grid, scoring="accuracy", n_iter=50, cv=cv)
    randomized_search.fit(X_train, Y_train)
    best_params_rfe = randomized_search.best_params_
    
    # Best parameters based on 20 repetitions
    # best_params_rfe = {'alpha': np.float64(0.3384698508856418), 'colsample_bytree': np.float64(0.7005590471231655), 'gamma': np.float64(0.14738669475440902), 'lambda': np.float64(0.5690549590459144), 'learning_rate': np.float64(0.023898523910037408), 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 82, 'scale_pos_weight': np.float64(2.3661971830985915), 'scoring_metric': make_scorer(f1_score, response_method='predict'), 'subsample': np.float64(0.9506076926900313)}
    
    print(best_params_rfe)

    xgb_clf_best = XGBClassifier(**best_params_rfe,enable_categorical=True, objective="binary:logistic")
    xgb_clf_best.fit(X_train_rfe, Y_train)

    explainer = shap.TreeExplainer(xgb_clf_best)
    shap_values = explainer(np.array(X_test_rfe))

    y_pred_rfe = xgb_clf_best.predict(X_test_rfe)
    print("Accuracy:", accuracy_score(Y_test, y_pred_rfe))
    print("Precision:", precision_score(Y_test, y_pred_rfe))
    print("Recall:", recall_score(Y_test, y_pred_rfe))
    print("F1:", f1_score(Y_test, y_pred_rfe))
    
    cm = confusion_matrix(Y_test, y_pred_rfe)
    tn, fp, fn, tp = cm.ravel()
    false_positive_rate = fp / (fp + tn)
    false_negative_rate = fn / (fn + tp)
    print("Confusion Matrix:\n", cm)
    print(f"False Positive Rate: {false_positive_rate:.2f}")
    print(f"False Negative Rate: {false_negative_rate:.2f}")

    removed_features2 = [feature_names[i] for i in range(len(feature_names)) if not rfecv.support_[i]]  
    print(f'Removed features: {removed_features2}')
    
    return xgb_clf_best

### Model verification

#### Curves

In [6]:
def learning_curve_chart():
    global scoring_metric, train_sizes, train_scores, test_scores
    train_sizes, train_scores, test_scores = learning_curve(
        xgb_clf_best, X_train_rfe, Y_train, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10), scoring=scoring_metric
    )

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='r')
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color='g')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score')

    plt.title('Learning Curve')
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    plt.legend(loc='best')
    plt.savefig('cv-score.png', dpi=300)
    plt.show()

#### Feature importances

In [7]:
def feature_importance():
    selected_features = [feature for feature, selected in zip(feature_names, rfecv.support_) if selected]  
    selected_importances = xgb_clf_best.feature_importances_

    importance_df = pd.DataFrame({'Feature': selected_features, 'Importance': selected_importances})  
    importance_df = importance_df.sort_values(by='Importance', ascending=False)  
      
    plt.figure(figsize=(12, 8))  
    plt.barh(importance_df['Feature'], importance_df['Importance'])  
    plt.xlabel('Importance')  
    plt.ylabel('Feature')  
    plt.title('Feature Importance from XGBoost')  
    plt.gca().invert_yaxis()  
    plt.savefig('feature_importance.jpg')
    plt.show()

#### SHAP

In [8]:
def shap_chart():
    selected_features = [feature for feature, selected in zip(feature_names, rfecv.support_) if selected]  
    shap.summary_plot(shap_values, X_test_rfe, feature_names=selected_features, show=False)  

    plt.savefig('shap.jpg')
    plt.show()

#### Confusion Matrix

In [9]:
def cm_chart():
    plt.figure(figsize=(8, 6))  
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['True','False'], yticklabels=['True','False'])  
    plt.xlabel('Predicted')  
    plt.ylabel('Actual')  
    plt.title('Confusion Matrix') 
    plt.savefig('confusion.jpg') 
    plt.show() 

### Model

In [ ]:
imbalanced_data = 1
scoring_metric = make_scorer(f1_score)

print(scoring_metric)

types_loading(df)

final_model = xgb_rfecv(scoring_metric, imbalanced_data)

learning_curve_chart()
feature_importance()   
shap_chart()
cm_chart()

#### Save model

In [115]:
file_name = "xgb_reg.pkl"

pickle.dump(final_model, open(file_name, "wb"))
